In [1]:
push!(LOAD_PATH, "../src")
using Quaternions
using FixedSizeArrays
using RigidBodyDynamics

In [2]:
mechanism = Mechanism{Float64}("world");
world = root_body(mechanism)

RigidBodyDynamics.RigidBody{Float64}(RigidBodyDynamics.CartesianFrame3D("world"),#undef)

In [3]:
body1 = RigidBody(rand(SpatialInertia{Float64}, CartesianFrame3D("body1")));
joint1 = Joint("1", QuaternionFloating());
joint1ToWorld = rand(Transform3D{Float64}, joint1.frameBefore, world.frame)
body1ToJoint1 = rand(Transform3D{Float64}, body1.frame, joint1.frameAfter)
attach!(mechanism, world, joint1, joint1ToWorld, body1, body1ToJoint1);

In [4]:
joint2 = Joint("2", Revolute(rand(Vec{3, Float64})));
joint2ToBody1 = rand(Transform3D{Float64}, joint2.frameBefore, body1.frame)
body2frame = CartesianFrame3D("b2")
body2 = RigidBody(rand(SpatialInertia{Float64}, body2frame));
body2ToJoint2 = rand(Transform3D{Float64}, body2frame, joint2.frameAfter)
attach!(mechanism, body1, joint2, joint2ToBody1, body2, body2ToJoint2);

In [5]:
x = MechanismState{Float64}(mechanism);

In [6]:
rand!(x)
x.v

DataStructures.OrderedDict{RigidBodyDynamics.Joint,Array{Float64,1}} with 2 entries:
  RigidBodyDynamics.Joint… => [0.4157326028957369,0.08678770191840113,0.9656418…
  RigidBodyDynamics.Joint… => [0.27653360147082906]

In [7]:
num_positions(x)

8

In [8]:
num_velocities(x)

7

In [9]:
cache = MechanismStateCache(mechanism, x);

RigidBodyDynamics.CartesianFrame3D("world")
RigidBodyDynamics.CartesianFrame3D("after_1")


In [10]:
relative_transform(cache, joint2.frameAfter, body1.frame)

RigidBodyDynamics.Transform3D{Float64}(RigidBodyDynamics.CartesianFrame3D("after_2"),RigidBodyDynamics.CartesianFrame3D("body1"),0.3743971772864054 + 0.16306002611445225im - 0.7479734090317736jm - 0.5232341358368291km,Vec(0.6611899206086929,0.6506304692427374,0.5943160045488547))

In [11]:
center_of_mass(cache)

RigidBodyDynamics.Point3D{Float64}(RigidBodyDynamics.CartesianFrame3D("world"),Vec(0.7151019706142665,0.7552160466711582,2.127093765385052))

In [12]:
mass(mechanism)

1.0874598613860254

In [13]:
transform(cache, Point3D{Float64}(joint2.frameAfter), root_frame(mechanism))

RigidBodyDynamics.Point3D{Float64}(RigidBodyDynamics.CartesianFrame3D("world"),Vec(0.3415857776127126,0.41975469147492467,1.9452876498921996))

In [14]:
relative_twist(cache, body2, body1)

RigidBodyDynamics.Twist{Float64}(RigidBodyDynamics.CartesianFrame3D("after_2"),RigidBodyDynamics.CartesianFrame3D("after_1"),RigidBodyDynamics.CartesianFrame3D("world"),Vec(0.2935334309354642,-0.2394150764437052,-0.18371591991435632),Vec(0.38861557212125397,0.6337617033930236,-0.20499281979912193))

In [15]:
relative_twist(cache, body1, world) + relative_twist(cache, body2, body1) + relative_twist(cache, world, body2)

RigidBodyDynamics.Twist{Float64}(RigidBodyDynamics.CartesianFrame3D("world"),RigidBodyDynamics.CartesianFrame3D("world"),RigidBodyDynamics.CartesianFrame3D("world"),Vec(0.0,0.0,0.0),Vec(0.0,0.0,0.0))

In [16]:
geometric_jacobian(cache, path(mechanism, body2, world))

RigidBodyDynamics.GeometricJacobian{Float64}(RigidBodyDynamics.CartesianFrame3D("world"),RigidBodyDynamics.CartesianFrame3D("after_2"),RigidBodyDynamics.CartesianFrame3D("world"),6x7 Array{Float64,2}:
 -1.06147    0.136693   0.831884  -0.537852  -0.0       -0.0       -0.0     
  0.865772  -0.829141  -0.201026  -0.521646  -0.0       -0.0       -0.0     
  0.664353   0.542071  -0.51726   -0.66227   -0.0       -0.0       -0.0     
 -1.40531    0.368745   0.13478    0.302177   0.136693   0.831884  -0.537852
 -2.29181    0.289835   0.195415  -0.535991  -0.829141  -0.201026  -0.521646
  0.741294   0.350341   0.140815   0.176773   0.542071  -0.51726   -0.66227 )

In [17]:
mass_matrix(cache)

7x7 Array{Float64,2}:
  0.0888495   1.27143   -0.483932   0.71467    …   0.0172074    -1.05497  
 -0.967504    0.136159  -0.271664   0.0889303     -0.0223487     0.174721 
  1.57325     0.69662   -0.46649    0.231507       0.724963     -0.257302 
  2.48514     0.446724  -0.636957   0.0            0.332218     -0.577589 
  0.243754    1.1396    -0.358901   0.864842       0.255792     -0.742673 
 -0.124586   -0.620914   0.682657  -0.332218   …  -1.38778e-17   0.338748 
 -1.05497     0.174721  -0.257302  -0.577589       0.338748      0.0868594

In [18]:
A = momentum_matrix(cache)

RigidBodyDynamics.MomentumMatrix{Float64}(RigidBodyDynamics.CartesianFrame3D("world"),6x7 Array{Float64,2}:
  0.727296  -0.0430824   0.118205   …  -0.385405    0.249182   -0.0183651
 -0.134834   0.759974   -0.0785557      0.0931337   0.241674   -0.455277 
 -0.38964   -0.364641   -0.0341227      0.239642    0.306824    0.563965 
 -0.609485  -1.34304     0.7528        -0.0193602  -0.258201    1.09656  
  2.04454    0.27559    -0.24417       -0.850508    0.326876   -0.202993 
 -1.30355   -0.0638976   0.611734   …   0.299402   -0.0477748  -0.56718  )

In [19]:
A * velocity_vector(x)

RigidBodyDynamics.Momentum{Float64}(RigidBodyDynamics.CartesianFrame3D("world"),Vec(0.4577122287865748,0.38326267874726155,0.012202331060566146),Vec(-0.38154343136840985,0.6373538721646081,0.13113801854883278))

In [20]:
accel = SpatialAcceleration(joint2.frameAfter, world.frame, joint2.frameAfter, rand(Vec{3, Float64}), rand(Vec{3, Float64}))

RigidBodyDynamics.SpatialAcceleration{Float64}(RigidBodyDynamics.CartesianFrame3D("after_2"),RigidBodyDynamics.CartesianFrame3D("world"),RigidBodyDynamics.CartesianFrame3D("after_2"),Vec(0.16005166227615897,0.07469536908807295,0.058754974180277975),Vec(0.408292788338007,0.6767399335636413,0.013577347611959167))

In [21]:
transform(cache, accel, world.frame)

RigidBodyDynamics.SpatialAcceleration{Float64}(RigidBodyDynamics.CartesianFrame3D("after_2"),RigidBodyDynamics.CartesianFrame3D("world"),RigidBodyDynamics.CartesianFrame3D("world"),Vec(0.120680335822923,-0.0525421525131535,-0.13161936979959019),Vec(0.8015072390368274,0.0724073001902881,-0.18062149944807826))